# Overlay viewer: PET / CT_1 / CT_2 (individual + combined)

This notebook discovers DICOM series in ../WholePelvis, reads three candidate series (CT_1, CT_2, PET),
shows each one by itself (center slice and a small montage), resamples the moving volumes to the CT_1 grid,
and displays overlay comparisons (combined and side-by-side).

How to run:
1) Install dependencies if missing: `pip install SimpleITK numpy matplotlib pydicom`
2) Open and run the cells in order.
3) Override the selected SeriesInstanceUID variables if you want specific series.

In [ ]:
# Imports and SimpleITK availability check
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
try:
    import SimpleITK as sitk
except Exception as e:
    sitk = None
    print('SimpleITK not available. Install with: pip install SimpleITK')
    print('Import error:', e)
# pydicom used only for fallback series discovery if sitk is missing
try:
    import pydicom
except Exception:
    pydicom = None

In [ ]:
# Helper functions: discover series, read a series by UID, resample, and display helpers
from collections import Counter

    data_dir = Path(data_dir)
    series_info = []
    if not data_dir.exists():
        return series_info
    if sitk is not None:
        sids = sitk.ImageSeriesReader.GetGDCMSeriesIDs(str(data_dir)) or []
        for sid in sids:
            files = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(str(data_dir), sid)
            series_info.append((sid, len(files), Path(files[0]) if files else None))
        return sorted(series_info, key=lambda x: -x[1])
    # fallback using pydicom to inspect SeriesInstanceUID per file (slower)
    if pydicom is None:
        return []
    files = list(data_dir.glob('*.dcm'))
    uids = Counter()
    first_file_for_uid = {}
    for i, f in enumerate(files):
        try:
            ds = pydicom.dcmread(str(f), stop_before_pixels=True, force=True)
            sid = getattr(ds, 'SeriesInstanceUID', 'UNKNOWN')
            uids[sid] += 1
            if sid not in first_file_for_uid:
                first_file_for_uid[sid] = f
        except Exception:
            uids['READ_ERROR'] += 1
    for sid, cnt in uids.most_common():
        series_info.append((sid, cnt, first_file_for_uid.get(sid, None)))
    return series_info

def read_series_by_id(data_dir, series_id):
    "Read a DICOM series specified by SeriesInstanceUID and return a SimpleITK image and filelist."
    if sitk is None:
        raise RuntimeError('SimpleITK required to read series')
    files = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(str(data_dir), series_id)
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(files)
    img = reader.Execute()
    return img, files

def resample_to_target(moving, fixed, transform=None, interpolator='linear', default_value=0.0):
    if sitk is None:
        raise RuntimeError('SimpleITK required for resampling')
    if transform is None:
        transform = sitk.Transform()
    interp_map = { 'linear': sitk.sitkLinear, 'nearest': sitk.sitkNearestNeighbor, 'bspline': sitk.sitkBSpline }
    sitk_interp = interp_map.get(interpolator, sitk.sitkLinear)
    resampled = sitk.Resample(moving, fixed, transform, sitk_interp, default_value, moving.GetPixelID())
    return resampled

def get_center_slice_indices(img, n=3):
    "Return up to n center-neighbour slice indices (array order)"
    size = img.GetSize()  # (x,y,z)
    nz = size[2]
    center = nz // 2
    # choose center +/- offsets (clamped)
    offsets = [0]
    if n > 1:
        offsets = [0] + [i for i in range(1, (n+1)//2)] + [-i for i in range(1, n//2+1)]
    idxs = sorted(set([min(max(center + o, 0), nz-1) for o in offsets]))
    return idxs[:n], nz

def show_image_slices(img, title=None, n=3, cmap='gray'):
    arr = sitk.GetArrayFromImage(img)  # array order (z,y,x)
    idxs, nz = get_center_slice_indices(img, n=n)
    cols = len(idxs)
    fig, axes = plt.subplots(1, cols, figsize=(3*cols, 3))
    if cols == 1:
        axes = [axes]
    for ax, k in zip(axes, idxs):
        sl = arr[k,:,:]
        disp = (sl - sl.min()) / max(1e-6, sl.max() - sl.min())
        ax.imshow(disp, cmap=cmap)
        ax.set_title(f'k={k}')
        ax.axis('off')
    if title:
        fig.suptitle(title)
    plt.tight_layout()
    plt.show()

def make_combined_overlay(base_img, overlays, overlay_cmaps=None, overlay_alphas=None, slice_k=None):
    "Show a single combined overlay for a chosen slice index (array-order)."
    base_arr = sitk.GetArrayFromImage(base_img)
    if slice_k is None:
        slice_k = get_center_slice_indices(base_img, n=1)[0][0]
    base = base_arr[slice_k,:,:].astype(float)
    base_disp = (base - base.min()) / max(1e-6, base.max() - base.min())
    fig, ax = plt.subplots(1,1, figsize=(8,8))
    ax.imshow(base_disp, cmap='gray')
    if overlay_cmaps is None:
        overlay_cmaps = ['hot', 'winter', 'viridis'][:len(overlays)]
    if overlay_alphas is None:
        overlay_alphas = [0.5]*len(overlays)
    for i, img in enumerate(overlays):
        arr = sitk.GetArrayFromImage(img)[slice_k,:,:].astype(float)
        disp = (arr - arr.min()) / max(1e-6, arr.max() - arr.min())
        ax.imshow(disp, cmap=overlay_cmaps[i%len(overlay_cmaps)], alpha=overlay_alphas[i%len(overlay_alphas)])
    ax.set_title(f'Combined overlay, slice k={slice_k}')
    ax.axis('off')
    plt.show()

In [ ]:
# Discover series in ../WholePelvis and print top candidates
data_dir = Path('..') / 'WholePelvis'
print('Looking in', data_dir.resolve())
series_info = discover_series(data_dir)
,
    CT1_SERIES_ID = series_info[0][0]
    # If we have >1 series, pick next largest as MOVING (CT_2)
    if len(series_info) > 1:
        MOVING_SERIES_ID = series_info[1][0]
    # try to find a PET-like series by filecount heuristics (e.g., mid-size)
    for sid, cnt, ex in series_info:
        if sid != CT1_SERIES_ID and sid != MOVING_SERIES_ID and cnt >= 50:
            PET_SERIES_ID = sid
            break
# Print selections and allow user to edit these variables manually before running next cells
print('Selected CT_1 (target):', CT1_SERIES_ID)
print('Selected MOVING (CT_2):', MOVING_SERIES_ID)
print('Selected PET (optional):', PET_SERIES_ID)
: 
,
: {
: 

: [
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
,
3
,
,
,
,
,

: {
: {
: 
3
,
: 
,
: 

: {
: 
,
: 
3

: 4,
: 2